Ilopoihsh tou Random Forest

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

def load_word(M=4000,N=0):
  #M = Most used words
  #N = Least used words
  
  if(M>0 and N>0):
    return("You cand give in both parameters")
  if(M>0):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=M)
  if(N>0):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000-N)


  word_index = tf.keras.datasets.imdb.get_word_index()
  index2word = dict((i + 3, word) for (word, i) in word_index.items())
  index2word[0] = '[pad]'
  index2word[1] = '[bos]'
  index2word[2] = '[oov]'
  x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
  x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

  vocabulary = list()
  for text in x_train:
    tokens = text.split()
    vocabulary.extend(tokens)

  vocabulary = set(vocabulary)
  print(len(vocabulary))



  x_train_binary = list()
  x_test_binary = list()

  for text in tqdm(x_train):
    tokens = text.split()
    binary_vector = list()
    for vocab_token in vocabulary:
      if vocab_token in tokens:
        binary_vector.append(1)
      else:
        binary_vector.append(0)
    x_train_binary.append(binary_vector)

  x_train_binary = np.array(x_train_binary)

  for text in tqdm(x_test):
    tokens = text.split()
    binary_vector = list()
    for vocab_token in vocabulary:
      if vocab_token in tokens:
        binary_vector.append(1)
      else:
        binary_vector.append(0)
    x_test_binary.append(binary_vector)

  x_test_binary = np.array(x_test_binary)

  return x_train_binary,x_test_binary,y_train,y_test, x_train




In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import numpy as np 
from collections import Counter

def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]

def vote(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common

class RandomForest:
    def __init__(self, n_trees=10):
        self.n_trees = n_trees
    
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):

            tree=ID3(50)
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(x_train_binary[:int(len(X)*0.3)],y_train[:int(len(y)*0.3)])
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [vote(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)


def accuracy(y_true, y_pred):   
    accuracy = np.sum(y_true == y_pred)/len(y_true)   
    return accuracy


In [ ]:
x_train_binary,x_test_binary,y_train,y_test, x_train=load_word()

In [ ]:
from sklearn.metrics import classification_report
#mabe change the train name 
def learning_curve(x_binary,y,percentage,num_of_trees=3):
  x_part=x_binary[:int(len(x_binary)*percentage)]
  y_part=y[:int(len(y)*percentage)]
  rf=RandomForest(n_trees = num_of_trees)
  rf.fit(x_train_binary[:int(len(x_binary)*percentage)],y_train[:int(len(y)*percentage)])
  print(classification_report(y_part, rf.predict(x_part), labels=[0, 1]))



learning_curve(x_train_binary,y_train,0.1)
learning_curve(x_train_binary,y_train,0.2)
learning_curve(x_train_binary,y_train,0.3)
learning_curve(x_train_binary,y_train,0.4)
learning_curve(x_train_binary,y_train,0.5)
learning_curve(x_train_binary,y_train,0.6)
learning_curve(x_train_binary,y_train,0.7)
learning_curve(x_train_binary,y_train,0.8)
learning_curve(x_train_binary,y_train,0.9)
learning_curve(x_train_binary,y_train,1)

In [ ]:
learning_curve(x_test_binary,y_test,0.1)
learning_curve(x_test_binary,y_test,0.2)
learning_curve(x_test_binary,y_test,0.3)
learning_curve(x_test_binary,y_test,0.4)
learning_curve(x_test_binary,y_test,0.5)
learning_curve(x_test_binary,y_test,0.6)
learning_curve(x_test_binary,y_test,0.7)
learning_curve(x_test_binary,y_test,0.8)
learning_curve(x_test_binary,y_test,0.9)
learning_curve(x_test_binary,y_test,1)

# Ilopoihsh tou ID3:

In [ ]:
class Word:

  def __init__(self, value:str, index:int):
    self.value = value
    self.index = index
    self.ig = None
    self.entropy0 = None
    self.entropy1 = None
  
  def setIg(self, ig):
    if(ig != None):
      self.ig = float(ig)
      
  def getIg(self):
    return self.ig

  def setEntropy0(self, entropy0):
    self.entropy0 = entropy0

  def setEntropy1(self, entropy1):
    self.entropy1  = entropy1

  def __str__(self):
    return self.value

  def __repr__(self):
    return self.value + " " + str(self.ig)
  

In [ ]:
class Node:
  value: Word
  left = None # must be Node
  right = None # must be Node
  depth = None
  p0 = None
  p1 = None

  def __init__(self, v: int = None, depth = None):
    self.value = v
    if depth == None:
      self.depth = 10
    else:
      self.depth = depth
  
  def setLeftChild(self, v: Word):
    self.left = v

  def setRightChild(self, v: Word):
    self.right  = v

  def setChilds(self, l, r):
    self.setLeftChild(l)
    self.setRightChild(r)

  def setP0(self, p0):
    self.p0 = p0
  
  def setP1(self, p1):
    self.p1 = p1

  def __str__(self):
    return self.value.value

In [ ]:
def calculate_entropy_word(binary_array, result_array, c, word, word_position, cProb):
  if(cProb == 1 or cProb == 0):
    return 0
  return - ( cProb * np.log2(cProb) ) - ( (1.0 - cProb) * np.log2(1.0 - cProb))


def calculate_entropy_total(binary_array, result_array):
  first = 0
  count = 0
  for element in binary_array:
      if(result_array[count] == 1):
        first += 1
      count += 1
  cProb = first/binary_array.shape[0]
  if(cProb == 1 or cProb == 0):
    return 0      
  return - ( cProb * np.log2(cProb) ) - ( (1.0 - cProb) * np.log2(1.0 - cProb))

In [ ]:
# -1 = has not found the word
def return_word(word):
  count = 0
  for element in vocabulary:
    if word == element:
      return count
    else:
      count += 1
  
  return -1

In [ ]:
def calculate_probability(array, result_array, c, word, word_position = -1):
  #Entropy
  has_the_word = 0 # c value
  count = 0 # lengeth
  is_positive = 0
  if word_position == -1:
    word_position = return_word(word) # the word column
  
  for element in array:
      element_i = element[word_position]
      count += 1
      if(element_i == c):
        has_the_word += 1
        if(result_array[count-1] == 1):
          is_positive += 1
  
  if(has_the_word == 0):
    return 0
  
  return is_positive/has_the_word

In [ ]:
def calculate_probability_total(result_array):
  #Node 
  count = 0
  p = 0
  for x in result_array:
    if(x == 1):
      p += 1
    count += 1
  return p/count

In [ ]:
def calculate_IG(binary_array, result_array, word, calculate_entrupy_total):
    word.setIg(calculate_entrupy_total - calculate_sum(binary_array, result_array, word))

In [ ]:
def hasTheWord(word1: Word, word2: Word, array):
  index1 = int(word1.index)
  index2 = int(word2.index)
  for x in array:
    if(x[index1] == 1 and x[index2] == 1):
      return 1
  return 0

In [ ]:
def remove_words(array, word_index, keep_the_word = True):
  num = []
  c = 1
  if(keep_the_word):
    c = 0
  count = 0
  for x in array:
    if(x[word_index] == c):
      num.append(count)
    count+=1

  return num

In [ ]:
def findValueFromArray(array, index):
  if len(array) < index:
    return None
  try:
    if(array[index] == 1):
     return index
  except:
    return None
  return None

In [ ]:
import math 
def get_my_key(obj:Word):
  return obj.getIg()

def next_word_subtree(array, result_array, words):
  i = 0
  et = calculate_entropy_total(array, result_array)
  words_to_remove = list()
  for word in words:
   word.setEntropy0(None)
   word.setEntropy1(None)
   word.setIg(None)
   calculate_IG(array, result_array, word, et)
  for x in words:
    if x.ig == None or math.isnan(x.ig):
     words_to_remove.append(x)
  for x in words_to_remove:
      words.remove(x)
  words = sorted(words, key=get_my_key, reverse=True)
  re = words.pop()
  return (re, words)



In [ ]:
def calculate_words():
  words = list()
  i = 0
  for x in vocabulary:
    if(x != '[pad]' and x!= '[bos]' and x != '[oov]'):
      words.append(Word(x, i))
    i+= 1

  return words

In [ ]:
def calculate_sum(binary_array, result_array, word):
    word_position = word.index
    p0 = (calculate_probability(binary_array, result_array, 0, word, word_position))
    p1 = 1 - p0
    word.setEntropy0(calculate_entropy_word(binary_array, result_array, 0, word,  word_position, p0))
    word.setEntropy1(calculate_entropy_word(binary_array, result_array, 1, word,  word_position, p1))
    if p0 == 0 or word.entropy0 ==0:
      a = 0
    else:
      a = np.log2(p0 * word.entropy0)

    if p1 == 0 or word.entropy1 ==0:
      b = 0
    else:
      b = np.log2(p1 * word.entropy1)
    return  a + b

In [ ]:
class ID3:
  def __init__(self, depth):
    self.tree = None
    self.depth = depth

  def anadromi(self, array, result_array, depth, words = None):
    if(words == None):
      words = calculate_words()
    if(len(array) == 0 or len(words) == 0 or depth == 0):
      return None
    data = next_word_subtree(array, result_array, words)
    word = data[0]
    words = data[1]
    node = Node(word, depth)
    node.setP1(calculate_probability_total(result_array))
    node.setP0(1-node.p1)
    l_num = remove_words(array, word.index, False)
    l_array = np.delete(array, l_num, 0)
    l_result_array = np.delete(result_array, l_num, 0)
    node.setLeftChild(self.anadromi(l_array, l_result_array, depth-1, words.copy()))
    r_num = remove_words(array, word.index)
    r_array = np.delete(array, r_num, 0)
    r_result_array = np.delete(result_array, r_num)
    node.setRightChild(self.anadromi(r_array, r_result_array, depth-1, words.copy()))
    return node

  def fit(self, array, result_array):
      self.tree = self.anadromi(array, result_array, self.depth)   

  def predict(self, table):
    good_result = 0
    count = 0
    results_array = []
    for sentense in table:
      tree = self.tree
      result = None
      while True:
        if tree.left == None and tree.right == None or tree.p1 == 1 or tree.p0 == 1:
          result = tree
          break
        else:
          sentense_word = findValueFromArray(sentense, tree.value.index)
          if(sentense_word == None):
            if tree.left == None:
              result = tree
              break
            tree = tree.left
          else:
            if tree.right == None :
              result = tree
              break 
            tree = tree.right
      c = 1
      if(result.p0 > result.p1):
        c = 0
      results_array.append(c)
    return results_array

In [ ]:
from sklearn.metrics import classification_report
#mabe change the train name 
def learning_curve(x_binary,y,percentage):
  x_part=x_binary[:int(len(x_binary)*percentage)]
  y_part=y[:int(len(y)*percentage)]
  nb=ID3(100)
  nb.fit(x_train_binary[:int(len(x_binary)*percentage)],y_train[:int(len(y)*percentage)])
  print(classification_report(y_part, nb.predict(x_part), labels=[0, 1]))